<a href="https://colab.research.google.com/github/cwattsnogueira/rating-predictor-spam-detection-review-summarizer/blob/main/13_04_review_summarization_gpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai gradio pandas --quiet

In [2]:
import pandas as pd
import openai
import gradio as gr
from google.colab import userdata

In [3]:
openai.api_key = userdata.get("OPENAI_API_KEY")

In [4]:
def clean_reviews(text_list):
    return [str(t).strip().replace("\n", " ") for t in text_list if isinstance(t, str) and len(t.strip()) > 0]

def summarize_with_gpt4(product_name, reviews):
    try:
        cleaned = clean_reviews(reviews)
        prompt = f"Summarize the following customer reviews for the product '{product_name}' in 3 sentences:\n" + " ".join(cleaned)
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        summary = response.choices[0].message.content
        return f"**Product:** {product_name}\n\n**Summary:**\n{summary}"
    except Exception as e:
        return f"Error during summary generation: {str(e)}"

In [5]:
def summarize_from_csv(product_name, file, column_name="review_text"):
    try:
        df = pd.read_csv(file.name)
        if column_name not in df.columns:
            return f"Column '{column_name}' not found in uploaded CSV."
        reviews = df[column_name].tolist()
        return summarize_with_gpt4(product_name, reviews)
    except Exception as e:
        return f"Error processing CSV: {str(e)}"

In [6]:
with gr.Blocks() as demo:
    gr.Markdown("##  Review Summarization App (GPT-4)")
    gr.Markdown("Upload a CSV or paste reviews manually. Include the product name to personalize the summary.")

    with gr.Tab("Manual Input"):
        product_name_manual = gr.Textbox(label="Product Name", placeholder="e.g., Wireless Headset")
        manual_input = gr.Textbox(lines=12, placeholder="Paste multiple reviews here...")
        manual_button = gr.Button("Summarize")
        manual_output = gr.Textbox(label="Summary", lines=12, max_lines=20)

    with gr.Tab("CSV Upload"):
        product_name_csv = gr.Textbox(label="Product Name", placeholder="e.g., Wireless Headset")
        csv_input = gr.File(label="Upload CSV")
        column_name = gr.Textbox(value="review_text", label="Column name with reviews")
        csv_button = gr.Button("Summarize CSV")
        csv_output = gr.Textbox(label="Summary", lines=12, max_lines=20)

    manual_button.click(
        lambda name, text: summarize_with_gpt4(name, text.split("\n")),
        inputs=[product_name_manual, manual_input],
        outputs=manual_output
    )

    csv_button.click(
        summarize_from_csv,
        inputs=[product_name_csv, csv_input, column_name],
        outputs=csv_output
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b4203bca8fbe64356.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
